## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-22-54-22 +0000,bbc,"UK armed forces helped US seizure of tanker, s...",https://www.bbc.com/news/articles/ckglnprzk72o...
1,2026-01-07-22-52-00 +0000,wsj,Opinion | Trump’s Venezuela Oil Play,https://www.wsj.com/opinion/donald-trump-venez...
2,2026-01-07-22-49-47 +0000,nyt,Trump Says He Wants Congress to Stop Wall St. ...,https://www.nytimes.com/2026/01/07/business/tr...
3,2026-01-07-22-48-00 +0000,wsj,Opinion | Judge Dugan Leaves the Bench,https://www.wsj.com/opinion/judge-hannah-dugan...
4,2026-01-07-22-47-00 +0000,wsj,Opinion | The Iranian Protests Escalate,https://www.wsj.com/opinion/iran-protests-ayat...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,54
8,venezuela,39
9,oil,27
27,maduro,20
312,new,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
40,2026-01-07-22-00-42 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,145
284,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,145
208,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,142
5,2026-01-07-22-46-52 +0000,nyt,"During Maduro Raid in Venezuela, a Close Call ...",https://www.nytimes.com/2026/01/07/us/politics...,134
68,2026-01-07-21-17-14 +0000,nyt,"After Maduro Ouster, Trump’s Venezuela Moves P...",https://www.nytimes.com/2026/01/07/us/politics...,128


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
40,145,2026-01-07-22-00-42 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...
94,103,2026-01-07-20-27-33 +0000,nypost,New Minneapolis ICE shooting video shows vehic...,https://nypost.com/2026/01/07/us-news/new-minn...
292,82,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
88,63,2026-01-07-20-42-33 +0000,nypost,Venezuelan interim leader Delcy Rodríguez fire...,https://nypost.com/2026/01/07/world-news/venez...
228,51,2026-01-07-13-34-42 +0000,cbc,U.S. Secretary of State Rubio says he will mee...,https://www.cbc.ca/news/world/us-greenland-eur...
11,44,2026-01-07-22-41-42 +0000,nyt,Venezuela Live Updates: U.S. Forces Seize Two ...,https://www.nytimes.com/live/2026/01/07/world/...
193,40,2026-01-07-16-22-02 +0000,bbc,Canada PM Mark Carney to visit China next week...,https://www.bbc.com/news/articles/cevnek9k32no...
309,36,2026-01-07-06-03-32 +0000,nypost,Venezuelan ‘dark fleet’ tanker evades US Coast...,https://nypost.com/2026/01/07/world-news/venez...
38,36,2026-01-07-22-02-00 +0000,wsj,Senate Republican negotiators say they are clo...,https://www.wsj.com/politics/policy/aca-subsid...
171,30,2026-01-07-17-37-25 +0000,nypost,"Girl, 16, killed in a freak four-wheeler accid...",https://nypost.com/2026/01/07/world-news/teen-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
